In [1]:
import json
import tensorflow as tf 
import sklearn
print(tf.__version__)


import os
import random
import sys

#%tensorflow_version 1.4 

## Package
import glob 
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import plotly.graph_objs as go
#import plotly.offline as py
#import plotly.tools as tls
import seaborn as sns
import scipy.io.wavfile
from sklearn.model_selection import StratifiedShuffleSplit
#py.init_notebook_mode(connected=True)

from scipy import signal
from tqdm import tqdm


from keras import regularizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
import keras
from keras import backend as K




new_labeled_data=pd.read_csv('./data2.csv')
X = new_labeled_data.drop(['label'], axis=1)
y = new_labeled_data.label
X_Stratified = StratifiedShuffleSplit(1, test_size=0.2, random_state=0)
for train_index, test_index in X_Stratified.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


x_train_cnn = np.expand_dims(X_train, axis=2)
x_test_cnn = np.expand_dims(X_test, axis=2)

x_train_cnn.shape


























model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.10))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
# Edit according to target class no.
model.add(Dense(16))
model.add(Activation(tf.nn.softmax))
opt = keras.optimizers.SGD(lr=0.01, momentum=0.5, decay=0.0, nesterov=False)




model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("./CNN3.h5")
print("Loaded model from disk")


new_model = keras.models.load_model('CNN3.h5')
##
#test_sound=one_sound('---.wav')
#a,b,c,d,e=feature_extraction(test_sound)
#test_f=stacking horizontal (a,b,c,d,e)
#loaded_model.predict(test_f)



# evaluate loaded model on test data
#loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#score = loaded_model.evaluate(x_test_cnn, y_test, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

#preds = loaded_model.predict(x_test_cnn, batch_size=16,
#                         verbose=1)

/Users/saurabh/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/saurabh/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/saurabh/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/saurabh/opt/anaconda3/lib/python3.7/site-packages/tensorf

1.14.0


Using TensorFlow backend.


(1878, 193)
(470, 193)
(1878, 16)
(470, 16)

Loaded model from disk



In [2]:
def feature_extraction(file_name):
    X, sample_rate = librosa.load(file_name)
    if X.ndim > 1:
        X = X[:,0]
    X = X.T
    
    # Get features   
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0) #40 values
    #zcr = np.mean(librosa.feature.zero_crossing_rate)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0) #tonal centroid features
    
    ##Return computed features
    return mfccs, chroma, mel, contrast, tonnetz


  

In [14]:
#data = pd.DataFrame(columns=['feature'])
audio_path='./sed_data_set/Actor_01/03-01-01-01-02-01-01.wav'
mfccs, chroma, mel, contrast, tonnetz = feature_extraction(audio_path)
extracted_features= np.hstack([mfccs,chroma, mel, contrast, tonnetz]) 
extracted_features_reshaped=extracted_features.reshape(1,193)

#from sklearn.externals 
#import joblib
#scaler = joblib.load("scaler.save") 
extracted_features=scaler.transform(extracted_features_reshaped)
extracted_features_n=extracted_features.reshape(1,193,1)
new_model.predict_classes(extracted_features_n)
#print(extracted_features_reshaped.shape)

array([14])

In [9]:
lb.inverse_transform([0, 1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15])

array(['female_angry', 'female_calm', 'female_disgust', 'female_fearful',
       'female_happy', 'female_neutral', 'female_sad', 'female_surprised',
       'male_angry', 'male_calm', 'male_disgust', 'male_fearful',
       'male_happy', 'male_neutral', 'male_sad', 'male_surprised'],
      dtype=object)